In [19]:
import pandas as pd

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
data = pd.read_csv("/content/drive/MyDrive/train_data.csv", on_bad_lines='skip')

In [24]:
model_name = "xlm-roberta-base"

In [25]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

Could not locate the tokenizer configuration file, will try to use the model config instead.


Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}



Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

loading file sentencepiece.bpe.model from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/sentencepiece.bpe.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_r

In [26]:
#preprocessing
def preprocessing(data):
  import nltk
  from nltk.stem.porter import PorterStemmer
  import re
  from nltk.corpus import stopwords
  ps = PorterStemmer()
  sentences = []
  nltk.download("stopwords")

  for i in range(data.shape[0]): # data.shape[0]
    sentence = re.sub('[^a-zA-Z]',' ',data.iloc[i].values[0]) # noktalama işaretlerini silme.
    sentence = sentence.lower() # kelimelerin hepsini küçük harf 
    sentence = sentence.split() # keliemeleri listeye atma
    sentence = [ps.stem(kelime) for kelime in sentence if not kelime in set(stopwords.words("english"))] #anlamı olmayan(stopwords) kelimeleri listeden atma
    sentence = " ".join(sentence) # listede kalan kelimeleri birleştirip yeni cümleyi oluşturma.
    sentences.append(sentence)
  df = pd.DataFrame(data = sentences,columns = ["Haber"])
  df["Truth"] = data["Truth"]
  return df

In [27]:
preprocessing_data = preprocessing(data)
preprocessing_data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Haber,Truth
0,undercov video prove trump right voter fraud n...,0
1,club face fa cup first round,1
2,world cup injuri watch doubt,1
3,statu world cup winner charlton unveil,1
4,polic apolog pursu sex assault report,1
...,...,...
42198,turkey close border iraq evacu embassi,1
42199,storm arwen damag tree use help save salmon,1
42200,iraq war veteran take cowardli republican wood...,0
42201,deplor hillari got bust offend half basecal be...,0


In [28]:
def process_data(row):
    sentence = row[0]
    truth_value = row[1]
    sentence = " ".join(sentence.split())
    encodings = tokenizer(sentence,padding = "max_length",truncation =True,max_length = 128)
    encodings["label"] = truth_value
    encodings["text"] = sentence
    return encodings

In [29]:
processed_data = []
for i in range(0,preprocessing_data.shape[0]):
    processed_data.append(process_data(preprocessing_data.iloc[i]))

In [30]:
veri = pd.DataFrame(data = processed_data)
veri

,attention_mask,input_ids,label,text
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1379, 13562, 1202, 23534, 69071, 7108, 169...",0,undercov video prove trump right voter fraud n...
1,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 11938, 2577, 1071, 45364, 5117, 68807, 2, ...",1,club face fa cup first round
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[0, 8999, 45364, 23, 461, 416, 39544, 92814, 2...",1,world cup injuri watch doubt
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 33908, 8999, 45364, 159690, 21441, 141, 15...",1,statu world cup winner charlton unveil
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 53087, 1747, 14071, 7398, 1159, 1100, 10, ...",1,polic apolog pursu sex assault report
...,...,...,...,...
42198,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[0, 2130, 19770, 20903, 132988, 17, 64020, 708...",1,turkey close border iraq evacu embassi
42199,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 77076, 187, 11697, 48, 15429, 53201, 4527,...",1,storm arwen damag tree use help save salmon
42200,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 17, 64020, 1631, 52129, 5646, 552, 19364, ...",0,iraq war veteran take cowardli republican wood...
42201,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 8, 254, 1484, 18995, 1703, 4163, 373, 271,...",0,deplor hillari got bust offend half basecal be...


In [31]:
from sklearn.model_selection import train_test_split
train_df,valid_df = train_test_split(veri,test_size = 0.2)

In [32]:
import pyarrow as pa
from datasets import Dataset
train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [33]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name) #,num_labels=2

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}



Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/pytorch_model.bin
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificat

In [34]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/Colab Notebooks/xlm-roberta-base/result",evaluation_strategy="epoch")
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer = tokenizer,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [35]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 33762
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12663
  Number of trainable parameters = 278045186
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, 

Epoch,Training Loss,Validation Loss
1,0.658600,0.707155
2,0.326400,1.564119
3,0.182600,0.156935


Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/xlm-roberta-base/result/checkpoint-500
Configuration saved in /content/drive/MyDrive/Colab Notebooks/xlm-roberta-base/result/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/xlm-roberta-base/result/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/xlm-roberta-base/result/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/xlm-roberta-base/result/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/xlm-roberta-base/result/checkpoint-1000
Configuration saved in /content/drive/MyDrive/Colab Notebooks/xlm-roberta-base/result/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/xlm-roberta-base/result/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab N

TrainOutput(global_step=12663, training_loss=0.4564914227398069, metrics={'train_runtime': 4030.3068, 'train_samples_per_second': 25.131, 'train_steps_per_second': 3.142, 'total_flos': 6662366588298240.0, 'train_loss': 0.4564914227398069, 'epoch': 3.0})

In [36]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8441
  Batch size = 8


{'eval_loss': 0.15693482756614685,
 'eval_runtime': 55.2049,
 'eval_samples_per_second': 152.903,
 'eval_steps_per_second': 19.129,
 'epoch': 3.0}

In [37]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/xlm-roberta-base/model")

Configuration saved in /content/drive/MyDrive/Colab Notebooks/xlm-roberta-base/model/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/xlm-roberta-base/model/pytorch_model.bin
